In [82]:
import pandas as pd
total = pd.read_csv("ffinal_total4.csv")

In [2]:
# required libraries
import numpy as np
np.random.seed(42)
import os
import urllib.request
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
import joblib

# machine learning libraries
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score
from scipy.stats import randint
# preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
# date
from datetime import date
import datetime 

In [83]:
total

,Unnamed: 0,기준년분기코드,행정동코드,월평균소득금액,음식지출총금액,유사업종점포수,개업점포수,폐업점포수,당월매출금액,총직장인구수,총상주인구수,총유동인구수,환산전체,환산1층,환산그외
0,0,20191,11290525,3664622,1011937000,90,2,4,4328200306,4429,12823,4444697,103305,119698,86911
1,1,20191,11200590,3654571,192261000,28,0,3,456182353,986,15796,4271714,126578,163240,89917
2,2,20191,11200520,3555413,300062000,48,2,0,1156401286,1218,18133,4486970,80865,109410,52321
3,3,20191,11170570,3955417,248116000,54,5,3,1103200412,5206,14933,3484892,79883,81935,77832
4,4,20191,11170510,3150859,548758000,86,0,1,3626619880,3345,18358,4805181,81171,111138,51203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7950,7950,20234,11710632,3910198,1526342000,115,6,6,8085087192,10171,31488,6197742,94031,121008,67053
7951,7951,20234,11650520,5196131,4127886000,262,14,9,31165275563,67858,17183,6791734,181696,215355,148038
7952,7952,20234,11680630,4101144,5519540000,304,15,10,43643536399,33305,20756,10010985,160349,184079,136619
7953,7953,20234,11650621,4557937,1474917000,119,0,4,11522249834,21837,22692,6565906,105925,131794,80055


In [84]:
total['기준년분기코드'].unique()

array([20191, 20192, 20193, 20194, 20201, 20202, 20203, 20204, 20211,
       20212, 20213, 20214, 20221, 20222, 20223, 20224, 20231, 20232,
       20233, 20234])

In [85]:
total.columns

Index(['Unnamed: 0', '기준년분기코드', '행정동코드', '월평균소득금액', '음식지출총금액', '유사업종점포수',
       '개업점포수', '폐업점포수', '당월매출금액', '총직장인구수', '총상주인구수', '총유동인구수', '환산전체',
       '환산1층', '환산그외'],
      dtype='object')

# 모델링

In [86]:
#train,test
y = total[['당월매출금액']]
X = total[[ '월평균소득금액', '음식지출총금액', '유사업종점포수', '총직장인구수', '총상주인구수', '총유동인구수', '환산전체',
       '환산1층', '환산그외']]

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
print("X_train: ", X_train.shape), print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape), print("y_test: ", y_test.shape)

X_train:  (6364, 9)
y_train:  (6364, 1)
X_test:  (1591, 9)
y_test:  (1591, 1)


(None, None)

In [88]:
#스케일링
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

print(X_scaled[:1])
print("----------------")
print(y_scaled[:1])

[[ 0.28052266 -0.28145204 -0.4546572  -0.31168161 -1.10531445 -0.42329121
  -0.25639578 -0.37747076 -0.062754  ]]
----------------
[[-0.30789788]]


In [89]:
# 데이터 분할 (훈련 세트와 테스트 세트)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# LightGBM

In [90]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

In [91]:
# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# LightGBM 데이터셋 생성
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# 하이퍼파라미터 설정
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',  # 회귀의 경우 'regression', 분류의 경우 'binary' 또는 'multiclass'
    'metric': 'rmse',  # 회귀의 경우 'rmse', 분류의 경우 'binary_logloss' 또는 'multi_logloss'
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'early_stopping_rounds' : 10
}

# 모델 학습
bst = lgb.train(
    params,
    train_data,
    num_boost_round=1000,
    valid_sets=[train_data, test_data])

# 예측 수행
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)
y_pred_mse = mean_squared_error(y_test, y_pred)
y_pred_rmse = np.sqrt(y_pred_mse)

y_train_pred = bst.predict(X_train, num_iteration=bst.best_iteration)
bst_mse = mean_squared_error(y_train, y_train_pred) 
bst_rmse = np.sqrt(bst_mse)


# R-squared (Coefficient of Determination) 계산
r2 = r2_score(y_test, y_pred)
print("R-squared (R2):", r2)
print("bst_rmse:", rmse)
print("-----------------------------------------------")
print("R2 score on training set : ", r2_score(y_train, y_train_pred))
print("R2 score on test set : ", r2_score(y_test, y_pred))

/home/encore/miniconda3/lib/python3.10/site-packages/lightgbm/basic.py:335: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000415 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2292
[LightGBM] [Info] Number of data points in the train set: 6364, number of used features: 9
[LightGBM] [Info] Start training from score 0.000492
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[832]	training's rmse: 0.0625287	valid_1's rmse: 0.154322
R-squared (R2): 0.9761983018271956
bst_rmse: 62361346210.091064
-----------------------------------------------
R2 score on training set :  0.9960895951658163
R2 score on test set :  0.9761983018271956


In [92]:
# LGBM 모델 초기화 및 훈련
import lightgbm as lgb
import joblib

# LightGBM 모델 초기화 및 훈련
lgb_model = lgb.LGBMRegressor(random_state=42, n_estimators=100)
lgb_model.fit(X_train, y_train.ravel())

# 예측
y_pred_scaled = lgb_model.predict(X_test)

# 역변환 (inverse transform)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))

# 모델 평가 (역변환된 값 사용)
mse = mean_squared_error(scaler_y.inverse_transform(y_test), y_pred)
r2 = r2_score(scaler_y.inverse_transform(y_test), y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000244 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2292
[LightGBM] [Info] Number of data points in the train set: 6364, number of used features: 9
[LightGBM] [Info] Start training from score 0.000492
Mean Squared Error: 3.126963901489062e+18
R-squared: 0.9665887308544976


In [93]:
# 최종 모델 저장
joblib.dump(lgb_model, 'Lightgbm.pkl', compress=1)

# 모델 로드
loaded_model = joblib.load('Lightgbm.pkl')

# 로드된 모델로 점수 계산
score = loaded_model.score(X_test, y_test)
print('test score: {score:.3f}'.format(score=score))

# 예측 결과 데이터프레임 생성
df = pd.DataFrame(scaler_X.inverse_transform(X_test), columns=X.columns)
predictions = scaler_y.inverse_transform(loaded_model.predict(X_test).reshape(-1, 1))
df['예측매출액'] = predictions

test score: 0.967


In [94]:
# 훈련된 데이터 프레임 타입 변경
df = df.astype(int)

In [95]:
print(df.isnull().sum())

월평균소득금액    0
음식지출총금액    0
유사업종점포수    0
총직장인구수     0
총상주인구수     0
총유동인구수     0
환산전체       0
환산1층       0
환산그외       0
예측매출액      0
dtype: int64


In [99]:
# 최초에 불러왔던 데이터 프레임과 머지
total_predict = df.merge(total, on = ['월평균소득금액', '음식지출총금액', '유사업종점포수', '총직장인구수', '총상주인구수', '총유동인구수', '환산전체',
       '환산1층', '환산그외'],how='inner')

In [100]:
# 중앙값 기준 설정 (테이블) 
median = total_predict.groupby('행정동코드')[['당월매출금액']].median().reset_index().rename(columns={'당월매출금액' : 'median'})
median

,행정동코드,median
0,11110515,2.111868e+09
1,11110530,2.987979e+10
2,11110540,5.392464e+09
3,11110550,1.131942e+09
4,11110560,2.531281e+09
...,...,...
389,11740640,5.544320e+09
390,11740650,6.665234e+09
391,11740660,6.704519e+09
392,11740685,1.521063e+10


In [102]:
final = median.merge(total_predict, on = ['행정동코드'],how = 'inner')
final

,행정동코드,median,월평균소득금액,음식지출총금액,유사업종점포수,총직장인구수,총상주인구수,총유동인구수,환산전체,환산1층,환산그외,예측매출액,Unnamed: 0,기준년분기코드,개업점포수,폐업점포수,당월매출금액
0,11110515,2.111868e+09,3780222,636006000,83,10628,12817,3612274,130611,165848,95374,3183396659,1618,20201,1,7,2061038996
1,11110515,2.111868e+09,3780222,936908000,84,10628,12434,3759011,125839,163704,87974,3430488045,3859,20212,3,6,2756910122
2,11110515,2.111868e+09,3780222,857237000,89,10628,12434,3595242,131721,183465,79977,3570972910,4085,20213,8,3,2381121646
3,11110515,2.111868e+09,3780222,787974000,84,10628,12817,3706408,131133,160242,102023,3389193878,2031,20202,1,0,2102339126
4,11110515,2.111868e+09,4015674,836137000,83,10997,13030,2552571,110935,144858,77012,4214094031,473,20192,2,2,2111867516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,11740700,5.425346e+09,3414406,875129000,113,4961,26347,6941531,76921,104173,49670,3778309876,3555,20211,2,2,4046773043
1459,11740700,5.425346e+09,3414406,1122368000,106,4961,26347,7182157,81769,113461,50077,4594637334,5200,20222,2,5,6441320449
1460,11740700,5.425346e+09,3414406,918173000,117,4961,27648,8380409,70079,91457,48702,4259803787,1934,20201,4,6,4288305740
1461,11740700,5.425346e+09,3414406,1191852000,104,4961,26347,7207926,93867,109900,77833,4880682087,5924,20223,2,4,6522555125


In [103]:
# 성공여부 컬럼 추가
final['success'] = final.apply(lambda row: 0 if row['median'] > row['예측매출액'] else 1, axis=1)

In [104]:
final

,행정동코드,median,월평균소득금액,음식지출총금액,유사업종점포수,총직장인구수,총상주인구수,총유동인구수,환산전체,환산1층,환산그외,예측매출액,Unnamed: 0,기준년분기코드,개업점포수,폐업점포수,당월매출금액,success
0,11110515,2.111868e+09,3780222,636006000,83,10628,12817,3612274,130611,165848,95374,3183396659,1618,20201,1,7,2061038996,1
1,11110515,2.111868e+09,3780222,936908000,84,10628,12434,3759011,125839,163704,87974,3430488045,3859,20212,3,6,2756910122,1
2,11110515,2.111868e+09,3780222,857237000,89,10628,12434,3595242,131721,183465,79977,3570972910,4085,20213,8,3,2381121646,1
3,11110515,2.111868e+09,3780222,787974000,84,10628,12817,3706408,131133,160242,102023,3389193878,2031,20202,1,0,2102339126,1
4,11110515,2.111868e+09,4015674,836137000,83,10997,13030,2552571,110935,144858,77012,4214094031,473,20192,2,2,2111867516,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,11740700,5.425346e+09,3414406,875129000,113,4961,26347,6941531,76921,104173,49670,3778309876,3555,20211,2,2,4046773043,0
1459,11740700,5.425346e+09,3414406,1122368000,106,4961,26347,7182157,81769,113461,50077,4594637334,5200,20222,2,5,6441320449,0
1460,11740700,5.425346e+09,3414406,918173000,117,4961,27648,8380409,70079,91457,48702,4259803787,1934,20201,4,6,4288305740,0
1461,11740700,5.425346e+09,3414406,1191852000,104,4961,26347,7207926,93867,109900,77833,4880682087,5924,20223,2,4,6522555125,0


In [105]:
final.to_csv("final_lightgbm.csv", index=False)

In [106]:
pd.read_csv("final_lightgbm.csv")

,행정동코드,median,월평균소득금액,음식지출총금액,유사업종점포수,총직장인구수,총상주인구수,총유동인구수,환산전체,환산1층,환산그외,예측매출액,Unnamed: 0,기준년분기코드,개업점포수,폐업점포수,당월매출금액,success
0,11110515,2.111868e+09,3780222,636006000,83,10628,12817,3612274,130611,165848,95374,3183396659,1618,20201,1,7,2061038996,1
1,11110515,2.111868e+09,3780222,936908000,84,10628,12434,3759011,125839,163704,87974,3430488045,3859,20212,3,6,2756910122,1
2,11110515,2.111868e+09,3780222,857237000,89,10628,12434,3595242,131721,183465,79977,3570972910,4085,20213,8,3,2381121646,1
3,11110515,2.111868e+09,3780222,787974000,84,10628,12817,3706408,131133,160242,102023,3389193878,2031,20202,1,0,2102339126,1
4,11110515,2.111868e+09,4015674,836137000,83,10997,13030,2552571,110935,144858,77012,4214094031,473,20192,2,2,2111867516,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,11740700,5.425346e+09,3414406,875129000,113,4961,26347,6941531,76921,104173,49670,3778309876,3555,20211,2,2,4046773043,0
1459,11740700,5.425346e+09,3414406,1122368000,106,4961,26347,7182157,81769,113461,50077,4594637334,5200,20222,2,5,6441320449,0
1460,11740700,5.425346e+09,3414406,918173000,117,4961,27648,8380409,70079,91457,48702,4259803787,1934,20201,4,6,4288305740,0
1461,11740700,5.425346e+09,3414406,1191852000,104,4961,26347,7207926,93867,109900,77833,4880682087,5924,20223,2,4,6522555125,0
